<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-cookbook/blob/main/notebooks/intent_class_synthetic/generating_test_scenarios.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



## Generate Test Scenarios and Evaluate Models

In [ ]:
OKAREO_API_KEY = "YOUR_API_KEY"

In [ ]:
%pip install okareo 

In [ ]:
# Import libraries
from okareo import Okareo
from okareo_api_client.models import ScenarioType
import os
import tempfile
import json
import pandas as pd

# Initialize Okareo client
okareo = Okareo(OKAREO_API_KEY)

# Load test data
test_data = pd.read_csv("blog_data/testing.csv")
rows = test_data.to_dict(orient="records")

# Write to a .jsonl file
temp_dir = tempfile.gettempdir()
file_path = os.path.join(temp_dir, "seed_data_sample.jsonl")
with open(file_path, "w+") as file:
    for row in rows:
        file.write(json.dumps(row) + '\n')
    

# Create scenario set with seed data file
source_scenario = okareo.upload_scenario_set(file_path=file_path, scenario_name="Blog Test Set")
print(source_scenario.app_link)

# make sure to clean up tmp file
os.remove(file_path)

### Generate Rephrased Questions

In [ ]:
# Use scenario set id or scenario set object from previous step as source for generation
rephrased_scenario = okareo.generate_scenarios(
    source_scenario=source_scenario,
    name="Blog - rephrase",
    number_examples=3,
    generation_type=ScenarioType.REPHRASE_INVARIANT
)

print(rephrased_scenario.app_link)

### Generate Misspellings

In [ ]:
spelling_scenario = okareo.generate_scenarios(
    source_scenario=source_scenario,
    name="Blog - spelling",
    number_examples=3,
    generation_type=ScenarioType.COMMON_MISSPELLINGS
)

print(spelling_scenario.app_link)

### Generate Contractions

In [ ]:
contr_scenario = okareo.generate_scenarios(
    source_scenario=source_scenario,
    name="Blog - contractions",
    number_examples=3,
    generation_type=ScenarioType.COMMON_CONTRACTIONS
)

print(contr_scenario.app_link)

### Generate Conditional Statements

In [ ]:
cond_scenario = okareo.generate_scenarios(
    source_scenario=source_scenario,
    name="Blog - conditional",
    number_examples=3,
    generation_type=ScenarioType.CONDITIONAL
)

print(cond_scenario.app_link)

### Evaluate the Base model on the synthetic data

This will create a classifier evaluation in Okareo

In [ ]:
# Load all of the necessary libraries from Okareo
from okareo.model_under_test import CustomModel, ModelInvocation

# Load the torch library
import torch

# Load libraries
from transformers import AutoTokenizer, DistilBertForSequenceClassification

# Load a tokenizer for the model from the Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Load your pretrained model from where it is stored
model = DistilBertForSequenceClassification.from_pretrained("blog_model_base")

# Define a model class that will be used used for classification
# The model takes in a scenario and returns a predicted class
class ClassificationModel(CustomModel):
    # Constructor for the model
    def __init__(self, name, tokenizer, model):
        self.name = name
        # The pretrained tokenizer
        self.tokenizer = tokenizer
        # The pretrained model
        self.model = model
        # The possible labels for the model
        self.label_lookup = ["pricing", "returns", "complaints"]

    # Callable to be applied to each scenario in the scenario set
    def invoke(self, input: str):
        # Tokenize the input
        encoding = self.tokenizer(input, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
        # Get the logits from the model
        logits = self.model(**encoding).logits
        # Get the index of the highest value (the predicted class)
        idx = torch.argmax(logits, dim=1).item()
        # Get the label for the predicted class
        prediction = self.label_lookup[idx]
        
        # Return the prediction in a ModelInvocation object
        return ModelInvocation(
                model_prediction=prediction,
                model_input=input,
                raw_model_output={ "prediction": prediction, "confidence": logits.softmax(dim=1).max().item() },
            )

# Register the model with Okareo
# This will return a model if it already exists or create a new one if it doesn't
model_under_test_base = okareo.register_model(name="blog_intent_classifier_model_base", model=ClassificationModel(name="Classification model", tokenizer=tokenizer, model=model), update=True)

#### Run the evaluations

In [ ]:
test_run_item = model_under_test_base.run_test(
    scenario=rephrased_scenario.scenario_id, 
    name="Blog - rephrase", 
    calculate_metrics=True)

test_run_item = model_under_test_base.run_test(
    scenario=spelling_scenario.scenario_id, 
    name="Blog - spelling", 
    calculate_metrics=True)

test_run_item = model_under_test_base.run_test(
    scenario=contr_scenario.scenario_id, 
    name="Blog - contractions", 
    calculate_metrics=True)

test_run_item = model_under_test_base.run_test(
    scenario=cond_scenario.scenario_id, 
    name="Blog - conditional", 
    calculate_metrics=True)

test_run_item = model_under_test_base.run_test(
    scenario=source_scenario.scenario_id, 
    name="Blog - base", 
    calculate_metrics=True)

### Now do the same thing with the Synthetic model

In [ ]:
# Load Okareo's pretrained model from the Hugging Face Hub
model = DistilBertForSequenceClassification.from_pretrained("blog_model_synthetic")

# Create an instance of the Okareo client
okareo = Okareo(OKAREO_API_KEY)

# Define a model class that will be used used for classification
# The model takes in a scenario and returns a predicted class
class ClassificationModel(CustomModel):
    # Constructor for the model
    def __init__(self, name, tokenizer, model):
        self.name = name
        # The pretrained tokenizer
        self.tokenizer = tokenizer
        # The pretrained model
        self.model = model
        # The possible labels for the model
        self.label_lookup = ["pricing", "returns", "complaints"]

    # Callable to be applied to each scenario in the scenario set
    def invoke(self, input: str):
        # Tokenize the input
        encoding = self.tokenizer(input, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
        # Get the logits from the model
        logits = self.model(**encoding).logits
        # Get the index of the highest value (the predicted class)
        idx = torch.argmax(logits, dim=1).item()
        # Get the label for the predicted class
        prediction = self.label_lookup[idx]
        
        # Return the prediction in a ModelInvocation object
        return ModelInvocation(
                model_prediction=prediction,
                model_input=input,
                raw_model_output={ "prediction": prediction, "confidence": logits.softmax(dim=1).max().item() },
            )

# Register the model with Okareo
# This will return a model if it already exists or create a new one if it doesn't
model_under_test_syn = okareo.register_model(name="blog_intent_classifier_model_w_synthetic", model=ClassificationModel(name="Classification model", tokenizer=tokenizer, model=model), update=True)

#### Run the evaluations

In [ ]:
test_run_item = model_under_test_syn.run_test(
    scenario=rephrased_scenario.scenario_id, 
    name="Blog - syn - rephrase", 
    calculate_metrics=True)

test_run_item = model_under_test_syn.run_test(
    scenario=spelling_scenario.scenario_id, 
    name="Blog - syn - spelling", 
    calculate_metrics=True)

test_run_item = model_under_test_syn.run_test(
    scenario=contr_scenario.scenario_id, 
    name="Blog - syn - contractions", 
    calculate_metrics=True)

test_run_item = model_under_test_syn.run_test(
    scenario=cond_scenario.scenario_id, 
    name="Blog - syn - conditional", 
    calculate_metrics=True)

test_run_item = model_under_test_syn.run_test(
    scenario=source_scenario.scenario_id, 
    name="Blog - syn - base", 
    calculate_metrics=True)